In [12]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import os
from importlib import reload

In [13]:
import sys
sys.path.append("..")

In [14]:
import geopandas as gpd
from rioxarray.exceptions import NoDataInBounds
import warnings

In [ ]:
from cryoswath import misc, l3, gis

In [ ]:
feather_list = glob.glob("*__l3_dfdump__*.feather", root_dir=misc.tmp_path)
feather_list

In [17]:
regions = [[region_str.split("_")[1] for region_str in [name_part for name_part in feather_name.split("__") if name_part.startswith("region")]][0] for feather_name in feather_list]
regions = [region for i, region in enumerate(regions) if region not in regions[:i]]

In [ ]:
for region_id in regions:
    ds = xr.open_zarr(os.path.join(misc.l3_path, f"{region_id}_monthly_500m.zarr"), decode_coords="all")
    for feather_name in glob.glob(f"*__l3_dfdump__region_{region_id}__*.feather", root_dir=misc.tmp_path):
        if not os.path.isfile(os.path.join(misc.tmp_path, feather_name)):
            print(f"Skipping {feather_name} because file not found. Was it processed already?")
            continue
        print(f"Processing {feather_name}")
        df = pd.read_feather(os.path.join(misc.tmp_path, feather_name))
        if any([len(df.index.levels[i])==1 for i in range(1, 3)]):
            warnings.warn(f"!! Singular on x or/and y !! Deleting file {feather_name} (anyway) !!")
        else:
            conti_time_index = pd.Index([(tix, df.index.levels[1][0], df.index.levels[2][0]) for tix in ds.time.values], name=df.index.names)
            df = df.reindex(index=df.index.join(conti_time_index, how="outer"))
            try:
                (l3.dataframe_to_rioxr(df, ds.rio.crs)
                    .rio.clip_box(*ds.rio.bounds())
                    .drop_vars(['spatial_ref'])
                    .to_zarr(os.path.join(misc.l3_path, f"{region_id}_monthly_500m.zarr"), region="auto"))
            except NoDataInBounds:
                warnings.warn(f"!! No data in bounds for {region_id} !! Deleting file {feather_name} (anyway) !!")
        print(f"... removing {feather_name}")
        os.remove(os.path.join(misc.tmp_path, feather_name))